<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Introduction to this part of the assignment - Geospatial data

In this work, after building a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, it will be converted addresses into their equivalent latitude and longitude values. 

## Libraries

In [1]:
import pandas as pd #library for data analsysis
import numpy as np #library to handle data in a vectorized manner

#!pip install geopy #instal geopy
from geopy.geocoders import Nominatim #library to convert an address into latitude and longitude values

#!pip install geocoder #instal geocoder
import geocoder #Library to handle gospatial data

import requests #library to handle requests
import json #library to handle JSON files
from pandas.io.json import json_normalize #library to tranform JSON file into a pandas dataframe

#Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!pip install folium #instal folium
import folium #map rendering library

import urllib.request #library to use to open URLs

from bs4 import BeautifulSoup #library to import the BeautifulSoup library so we can parse HTML and XML documents

print('Libraries imported.')


Libraries imported.


In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data_url = urllib.request.urlopen(url)
data_toronto = pd.read_html(data_url)
neighborhood = data_toronto[0]

In [3]:
#Droping "Not assigned" rows, they're not needed for analysis
neighborhood = neighborhood[neighborhood.Borough !='Not assigned']

#Grouping neighborhoods with the same postal code
neighborhood.set_index(['Postal Code','Borough'],inplace=True)
merge_neighborhood = neighborhood.groupby(level=['Postal Code','Borough'], sort=False).agg( ','.join)

In [4]:
# Resetting the index
new_neighborhood = merge_neighborhood.reset_index()

In [5]:
#Giving the borough's value to the neighborhood if is not assigned
new_name = new_neighborhood['Neighbourhood']
new_neighborhood[['Borough', 'Neighbourhood']]
borough_name = new_neighborhood['Borough']

for x in new_name:
    if x == 'Not assigned':
        y = borough_name
        z = new_neighborhood['Neighbourhood'].replace([x], [y])
        new_neighborhood['Neighbourhood'] = z
    else:
        None

In [6]:
neighborhood_all = pd.DataFrame(new_neighborhood)
neighborhood_all

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [7]:
neighborhood_all.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
print('Shape is:', neighborhood_all.shape)

Shape is: (103, 3)


## Latitude and Longitude

In [9]:
data_geo = pd.read_csv("https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv")
data_geo

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [10]:
postal_code = neighborhood_all['Postal Code']
postal_code

0      M3A
1      M4A
2      M5A
3      M6A
4      M7A
      ... 
98     M8X
99     M4Y
100    M7Y
101    M8Y
102    M8Z
Name: Postal Code, Length: 103, dtype: object

In [11]:
#Cancatenation of neighborhood_all and data_geo - Merging the two data sets over two columns on 'Postal Code'
merge_columns=pd.merge(neighborhood_all, data_geo, left_on='Postal Code', right_on='Postal Code')
toronto_lat_long = pd.DataFrame(merge_columns)
toronto_lat_long

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [12]:
toronto_lat_long.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [13]:
print('Shape is:', toronto_lat_long.shape)

Shape is: (103, 5)


In [1]:
# @hidden_cell
username = 'aaa'